# Normalisation des sous titres

Récupérer un fichier texte grâce à une regexp des fichiers. 

In [2]:
import spacy
import os
import re
import module_traitement as m
spacy.prefer_gpu()
nlp = spacy.load("fr_dep_news_trf")
from spacy.language import Language

In [3]:
# TEST - Pour Matignon - LSF 
file_with_path = m.lister_fichiers_with_path("../data/cr_audio_aligned/")
folder = m.lister_fichiers("../data/cr_audio_aligned/")
output_seg = "test_new_seg_cr/"
output_cleaning = "test_cleaning_cr/"
#output_sent = "../data/sentence_matignon/"

In [15]:
pattern = re.compile(r' [A-Z]\.')
for file,name in zip(file_with_path,folder):
    dict_sub = m.get_dict_vtt_clean(file)
    new_text = ""
    for k,v in dict_sub.items():
        for kk,vv in v.items():
            if kk == "text":
                new_text = new_text + vv.strip() + " "
    phrases = m.segmenter_texte_en_phrases(new_text)
    with open(f"try/{name}", 'w', encoding='utf-8') as f:
        for index, element in enumerate(phrases):
            f.write(element.strip())
            if index < len(phrases) - 1:
                f.write("\n")
